# Sparse stochastic variational inference

In [2]:
from tqdm import tqdm_notebook as tqdm
import autograd.numpy as np

# https://github.com/bamos/opt.py/blob/master/bamos_opt/pgd.py
def psgd(x0, g, proj, eps, maxit, lam, callback=None, **kwargs):
    """TODO."""
    results = {
        'feval': 0,
        'geval': 0
    }

    x = proj(np.copy(x0))

    for k in tqdm(range(maxit)):
        g_x = g(x, **kwargs)
        lam_val = lam(k)
        x = proj(x - lam_val * g_x)
        
        if callback:
            callback(k, x, g_x, lam_val)

    return x

def lam_inverse(init_lam):
    return lambda k: init_lam/k

def sort_rows(z):
    perm = anp.argsort(z, axis=1)
    rows = anp.arange(z.shape[0], dtype=int).reshape(-1, 1)
    return z[rows, perm]

def sparsemax_rows(z):
    ncols = z.shape[1]
    u = sort_rows(z)[:, ::-1]
    cssv = u.cumsum(axis=1) - 1
    ind = anp.arange(ncols) + 1
    cond = u - cssv / ind > 0
    argmin = anp.argmin(cond, axis=1)
    rho = np.where(argmin > 0, argmin, ncols)
    theta = cssv[anp.arange(z.shape[0]) , rho - 1]/rho
    return np.clip(z - theta.reshape(-1, 1), 0, None)

Always assume a dimension for samples

In [109]:
from autograd.scipy.stats import norm

n_clusters = 3
n_data = 4
mu = np.arange(n_clusters)
p = np.array([0.2, 0.3, 0.5])
indices = np.arange(n_clusters)
assignments = np.random.choice(indices, size=n_data, p=p)
assignments0 = np.random.choice(indices, size=n_data)
x = np.random.normal(loc=mu[assignments])
log_p = lambda assignments: np.sum(norm.logpdf(x[:, np.newaxis], loc=mu[assignments]) + np.log(p[assignments]), axis=0)
log_p(assignments[:, np.newaxis]), log_p(assignments0[:, np.newaxis])

(array([-9.99541264]), array([-13.54310484]))

In [118]:
lam0[assignments[:, np.newaxis]]

array([[[0.33333333, 0.33333333, 0.33333333]],

       [[0.33333333, 0.33333333, 0.33333333]],

       [[0.33333333, 0.33333333, 0.33333333]],

       [[0.33333333, 0.33333333, 0.33333333]]])

In [128]:
rows = np.arange(n_data)
log_q = lambda assignments, lam: np.log(lam[rows[:, np.newaxis], assignments]).sum(axis=0)
lam0 = np.ones((n_data, n_clusters))/n_clusters
log_q(assignments[:, np.newaxis], lam0), log_q(assignments0[:, np.newaxis], lam0)

(array([-4.39444915]), array([-4.39444915]))

In [131]:
kl_point = lambda assignments, lam: log_q(assignments, lam) - log_p(assignments)
kl_point(assignments[:, np.newaxis], lam0)

array([5.60096349])

In [132]:
def random_choice_row(p, n_samples=1):
    u = np.random.uniform(size=(p.shape[0], n_samples)) #[index, sample]
    pf = p.cumsum(axis=1) #[index, row]
    return np.argmax(u[:, np.newaxis, :] < pf[:, :, np.newaxis], axis=1)

random_choice_row(lam0), random_choice_row(lam0, n_samples=10)

(array([[2],
        [0],
        [1],
        [2]]), array([[0, 0, 1, 2, 2, 1, 1, 1, 0, 1],
        [2, 0, 2, 1, 2, 1, 2, 2, 1, 0],
        [1, 0, 0, 2, 0, 0, 2, 2, 1, 0],
        [1, 1, 2, 1, 0, 2, 0, 0, 2, 1]]))

In [133]:
def kl_mc(lam, n_samples=1):
    assignments = random_choice_row(lam, n_samples=n_samples)
    return np.mean(kl_point(assignments, lam)).sum()

kl_mc(lam0, n_samples=1000)

7.359771257322359